In [1]:
%load_ext autoreload
%autoreload 2
from hereutil import here, add_to_sys_path
add_to_sys_path(here())

from src.common_basis import *

eng, con = get_connection()

In [2]:
import pandas as pd
d = pd.read_sql("""SELECT `post_content`, `post_id`
FROM `dhh23`.`incel_posts_c`
WHERE (`post_content` REGEXP '(you|they|we)(\\'re| are) ')""", con)
d

,post_content,post_id
0,How the fuck they doxx so many people? Someone...,712305
1,For many years I killed lots of pretty animals...,671738
2,CuckTears hasn't touched this. \nThey know the...,712382
3,"they're pretty good, but pretty expensive tbh",712398
4,As a part of the 10% of this forum older than ...,671828
...,...,...
136730,\n\nuniversallyabhorred said:\n\n\n\nWhat's ne...,2266014
136731,\n\nuniversallyabhorred said:\n\n\n\nMy bad yo...,2266016
136732,\n\nWarmIncelation said:\n\n\n\nNot sure why y...,2266017
136733,\n\nuniversallyabhorred said:\n\n\n\nLooks lik...,2266018


In [6]:
import spacy
import re

from tqdm import tqdm
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
nlp.enable_pipe("senter")
matching_sentences = []
for doc, context in tqdm(nlp.pipe(map(lambda t: (re.sub("\\.?\\s*\\n+\\s*",". ",re.sub("Click to expand...",".", t[0])), t[1]), d.itertuples(index=False, name=None)), as_tuples=True, n_process=8), total=len(d)):
    for sentence in doc.sents:
        if re.search("(you|they|we)(\\'re| are)", sentence.text) is not None:
            matching_sentences.append((sentence.text, context))
matching_sentences

100%|██████████| 136735/136735 [03:10<00:00, 719.18it/s] 


[('Someone speaks against them, and boom they are doxxed.', 712305),
 ("Killing and eating varmints doesn't require a licence as long as they are destroying something.",
  671738),
 ('They know themselves we are harmless and sex havers are violent', 712382),
 ("they're pretty good, but pretty expensive tbh", 712398),
 ("I don't want to sound like I'm promoting drug use to incels, but if you're seriously considering suicide, at least consider slow suicide first.",
  671828),
 ('If you are going to give any "drug" a try other than weed I would recommend MDMA.',
  671828),
 ('During the time he beat me i told him you are beating me because of a tv and he said yes so what like it was justified.',
  671835),
 ("If you're underage GTFO. .", 671837),
 ("If you're underage GTFO. .", 671838),
 ("It's a good thing to avoid, if you're not ready to be killed or kill since any fight can lead to death.",
  671893),
 ("It's a good thing to avoid it, if you're not ready to be killed or kill since any 

In [7]:
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_trf", disable=["ner"])
docs = list(tqdm(nlp.pipe(matching_sentences, as_tuples=True, n_process=8), total=len(matching_sentences)))

100%|██████████| 169530/169530 [37:07<00:00, 76.12it/s] 


In [496]:
from spacy.tokens import DocBin
doc_bin = DocBin(docs=map(lambda t: t[0], docs))
doc_bin.to_disk(here("data/output/jiemakel/you_we_they_are.spacydocs"))

In [501]:
docs2 = list(doc_bin.from_disk(here("data/output/jiemakel/you_we_they_are.spacydocs")).get_docs(nlp.vocab))
docs2[1][0].pos_

'VERB'

In [492]:
for token in docs[99010][0]:
    if token.text=="are" or token.text=="'re":
        if token.dep_ == 'ROOT':
            print(list(map(lambda t: list(t.subtree), token.rights)))
        elif 'VerbForm=Part' in token.head.morph:
            print(token.head, list(next(token.head.rights).subtree))
        else:
            print(list(token.subtree))
#        print(list(map(lambda t: list(t.subtree), token.head.rights)))
        #for ot in token.subtree:
#            print(ot, ot.pos_)
 #           if ot.pos_=="ADJ" or ot.pos_=="VERB":
 #               print(ot)
        print(list(map(lambda t: list(t.subtree), token.head.lefts)),token.head,list(map(lambda t: list(t.subtree), token.head.rights)))

[that, you, 're, gay]
[[Just]] admit [[that, you, 're, gay], [,], [fam], [.], [.]]


In [361]:
from spacy import displacy
displacy.serve(docs[61][0], style='dep')




Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


'Life is good if you are not a mentalcel.'

In [ ]:
d['post_content']